# Welcome to the ADF.procfwk Handy User Guide Notebook

This Notebook provides a series of short code snippets and narrative to help developers work with processing framework metadata.

![alt text](https://mrpaulandrew.files.wordpress.com/2020/03/adfprocfwk-icon.png "ADF.procfwk Icon")

<span style="color:red">Note: do not just run all cells in this Notebook. This may make unwanted changes to your deployed solution. The EXEC code is written with example values for you to update.</span>

In [ ]:
SET NOCOUNT ON;

***
# Getting Current & Previous Execution Details

* The [CurrentExecution] table only contains data for processing runs that our in progress or in an incomplete state.

* The [ExecutionLog] table and [CompleteExecutionLog] views contains all other log details for execution runs.

In [ ]:
SELECT * FROM [procfwk].[CurrentExecution];

SELECT * FROM [procfwk].[CompleteExecutionLog];

SELECT * FROM [procfwk].[LastExecution];

SELECT TOP 100 * FROM [procfwk].[ExecutionLog] ORDER BY [LocalExecutionId], [StageId], [PipelineId], [StartDateTime];

EXEC [procfwk].[GetExecutionDetails];

***
# Getting Error Details

* The [CurrentExecution] table and the [CompleteExecutionErrorLog] view contain details of error logged for failed pipeline activities. A single pipeline can contain multiple activities that if executed in parallel may generate multiple errrors for a single pipeline Run ID.

In [ ]:
SELECT * FROM [procfwk].[ErrorLog];

SELECT * FROM [procfwk].[CompleteExecutionErrorLog];

EXEC [procfwk].[GetExecutionDetails];

***
# Reviewing Basic Processing Metadata
The following tables have been ordered as per there level within the processing framework. A logical hierarchy exists between these tables and is enforced via database constraints connecting primary and foreign keys.
* Data Factory's
* Stages
* Pipelines
* Pipeline Parameters

In [ ]:
SELECT * FROM [procfwk].[DataFactorys];

SELECT * FROM [procfwk].[Stages];

SELECT * FROM [procfwk].[Pipelines];

SELECT * FROM [procfwk].[PipelineParameters];

***
# Review and Check Logical Pipeline Metadata Integrity

The following procedure uses the optional attribute [LogicalPredecessorId] within the table [procfwk].[Pipelines] to create a chain of dependencies between processes. This chain has zero affect of the execution of the framework, but does allow data lineage chains to be reviewed and created without impacting processing.

The procedure also implements a series of case statement checks against pipeline chains to advise where metadata issues may exist. For example:

* Pipeline could be moved to an earlier stage if it has no predecessors and/or isn't in an earlier stage.
* Dependency issue, predeccessor pipeline is currently running in the same stage as successor.
* Disabled pipeline has downstream successors.
* Disabled stage has downstream successors.

It is recommended that these advisory points are reviewed and the logical chain of pipelines updated to inform better framework execution.



In [ ]:
EXEC [procfwk].[CheckStageAndPiplineIntegrity];

***
# Checking and Adding Framework Properties
* Use the [CurrentProperties] view to check only the latest version of property values.
* Use the complete [Properties] table to see all versions of framework properties with valid from and to dates.
* Use the stored procedure [AddProperty] to add a new property/value or update an existing property/value.
* Use the stored procedure [GetPropertyValue] to get the latest version of a single property value.

In [ ]:
SELECT * FROM [procfwk].[CurrentProperties];

SELECT * FROM [procfwk].[Properties];

EXEC [procfwk].[AddProperty] 
	@PropertyName = 'TenantId',
	@PropertyValue = '1234-1234-1234-1234-1234',
	@Description = 'Used to provide authentication throughout the framework execution.';

EXEC [procfwk].[GetPropertyValue]
	@PropertyName = N'TenantId';

***
# Checking, Getting, Adding and Deleting Service Principals

To **check** which SPN details are being used follow the guidance in the framework for pipeline authentication:

* The [ServicePrincipals] table in the [dbo] schema details which credentials are being used by the framework. These details are specific to an Azure Tenant. However, this table also needs to be joined with table [procfwk].[PipelineAuthLink] to better understand which credentials are being used by which Data Factory/Pipeline.

To **get** actual unencrypted SPN details the stored procedure [procfwk].[GetServicePrincipal].

Be careful not to change the Tenant ID property after Service Principals have been added.

To **add** Service Principals use the stored procedure [procfwk].[AddServicePrincipal]. This will handle the creation of the links between Data Factory's and Pipelines.

To **delete** Service Principals use the stored procedure [procfwk].[DeleteServicePrincipal]. This will also handle the removal of the links between Data Factory's and Pipelines.

Also, be aware that as part of the new metadata integrity checks introduced in v1.3 of the framework. This will ensure all enabled pipelines have a valid SPN before the execution run starts.

In [ ]:
--Checking:
SELECT * FROM [dbo].[ServicePrincipals];

SELECT
	SP.[PrincipalName],
	DF.[ResourceGroupName],
	DF.[DataFactoryName],
	PP.[PipelineName]
FROM
	[procfwk].[PipelineAuthLink] AL
	INNER JOIN [procfwk].[DataFactorys] DF
		ON AL.[DataFactoryId] = DF.[DataFactoryId]
	INNER JOIN [procfwk].[Pipelines] PP
		ON AL.[PipelineId] = PP.[PipelineId]
	INNER JOIN [dbo].[ServicePrincipals] SP
		ON AL.[CredentialId] = SP.[CredentialId];

--Getting:
EXEC [procfwk].[GetServicePrincipal]
	@DataFactory = 'FrameworkFactory',
	@PipelineName = 'Wait 1';

--Adding:
EXEC [procfwk].[AddServicePrincipal]
	@DataFactory = N'FrameworkFactory',
	@PrincipalId = N'1234-1234-1234-1234-1234',
	@PrincipalSecret = N'Passw0rd123!',
	@PrincipalName = N'ADFFrameworkExecutor',
	@SpecificPipelineName = N'Wait 1' --Optional parameter

--Deleteing:
EXEC [procfwk].[DeleteServicePrincipal]
	@DataFactory = N'FrameworkFactory',
	@PrincipalId = N'1234-1234-1234-1234-1234',
	@SpecificPipelineName = N'Wait 1' --Optional parameter

***
# Pre Execution Metadata Integrity Checks

This procedure now acts as a pass/fail check before a given framework exeuction runs starts. If any of the checks fail the Data Factory pipelines will not start and the execution run will be stalled.

In debug mode details of the checks that failed can be reviewed and fixed.


In [ ]:
EXEC [procfwk].[CheckMetadataIntegrity]
	@DebugMode = 1

***
# Resources and Content

| ![alt text](https://mrpaulandrew.files.wordpress.com/2020/03/azure-square-logo.png?w=75 "Blog Icon") | Blogs |[mrpaulandrew.com/ADF.procfwk](https://mrpaulandrew.com/category/azure/data-factory/adf-procfwk/)|
|:----:|:----:|:----:|
| ![alt text](https://mrpaulandrew.files.wordpress.com/2018/11/github-icon.png?w=75 "GitHub Icon") | **GitHub** |**[github.com/mrpaulandrew/ADF.procfwk](https://github.com/mrpaulandrew/ADF.procfwk)**  |
| ![alt text](https://mrpaulandrew.files.wordpress.com/2020/03/twitterlogo.png?w=75 "Twitter Icon") | **Twitter** |**[#ADFprocfwk](https://twitter.com/search?q=%23ADFprocfwk&amp;src=hashtag_click)** |